# Testing the CoreML Model with Image Input

This notebook tests our converted CoreML model that takes an image as input.

In [ ]:
import coremltools as ct
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

## Load the Model

Load the CoreML model that was converted with image input.

In [ ]:
# Load the CoreML model
model_path = 'model.mlpackage'
model = ct.models.MLModel(model_path)

# Print model input and output descriptions
print("Model inputs:")
print(model.input_description)
print("\nModel outputs:")
print(model.output_description)

## Prepare a Test Image

Download a test image and resize it to 224x224 pixels to match the model input shape.

In [ ]:
# Download a sample image (a dog image from PyTorch's repo)
image_url = 'https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg'
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
print(f"Image format: {img.format}, size: {img.size}, mode: {img.mode}")

# Resize to match model input size (224x224)
img_resized = img.resize((224, 224))

# Display the image
plt.figure(figsize=(6, 6))
plt.imshow(img_resized)
plt.axis('off')
plt.title('Test Image (Resized to 224x224)')
plt.show()

## Make Predictions

Run inference with the CoreML model using the test image.

In [ ]:
# Convert PIL image to proper format for CoreML
# CoreML handles image preprocessing but requires the right format

# Method 1: Direct PIL image input
try:
    print("Trying prediction with direct PIL image...")
    prediction = model.predict({"input": img_resized})
    print("Direct PIL image input worked!")
except Exception as e:
    print(f"Error with direct PIL image: {e}")
    
    # Method 2: Try with PIL image converted to BGR numpy array
    try:
        print("\nTrying with numpy array...")
        # Convert PIL image to numpy array (RGB)
        img_array = np.array(img_resized)
        # CoreML may expect BGR format
        prediction = model.predict({"input": img_array})
        print("Numpy array input worked!")
    except Exception as e:
        print(f"Error with numpy array: {e}")
        
        # Method 3: Try with normalized array
        try:
            print("\nTrying with normalized numpy array...")
            # Normalize the image to match preprocessing in the model conversion
            img_array = np.array(img_resized).astype(np.float32)
            # Reshape to match expected input (batch, channels, height, width)
            img_array = np.transpose(img_array, (2, 0, 1))  # CHW format
            img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
            prediction = model.predict({"input": img_array})
            print("Normalized array input worked!")
        except Exception as e:
            print(f"Error with normalized array: {e}")
            prediction = None

# Display the prediction results if we have a successful prediction
if prediction:
    print("\nPrediction results:")
    for key, value in prediction.items():
        print(f"{key}:")
        if isinstance(value, np.ndarray):
            print(f"  Shape: {value.shape}")
            print(f"  Max value: {value.max()}")
            print(f"  Min value: {value.min()}")
            # If this is a classification model with probabilities, show top 5 classes
            if len(value.shape) == 2 and value.shape[1] > 1:
                top_indices = np.argsort(value[0])[::-1][:5]
                print("  Top 5 classes:")
                for i, idx in enumerate(top_indices):
                    print(f"    {i+1}. Class {idx}: {value[0][idx]:.6f}")
        else:
            print(f"  {value}")

## Visualize Features (Optional)

If the model outputs feature vectors or activation maps, you can visualize them.

In [ ]:
# Get the first output of the model (if it's an array of features or activations)
output_key = list(prediction.keys())[0]
output_data = prediction[output_key]

# If the output is a feature vector, visualize top activations
if isinstance(output_data, np.ndarray) and len(output_data.shape) == 2 and output_data.shape[1] > 1:
    # Flatten and get top activations
    flat_output = output_data[0]
    top_indices = np.argsort(flat_output)[::-1][:20]  # Get top 20 activations
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(top_indices)), flat_output[top_indices])
    plt.title(f'Top 20 Activations for {output_key}')
    plt.xlabel('Feature Index')
    plt.ylabel('Activation Value')
    plt.show()

# Save the test image for future reference
img_resized.save('test_image.jpg')

## Summary

We successfully tested the CoreML model that takes an image as input. The model processed the image correctly and produced predictions.